In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from copy import copy
from labpythonlib.lab_functions import *
from tqdm import tqdm


import sys
sys.path.insert(1, '/home/emanuelsamir/Documentos/dev/concytec/journal_ws/src/motion_ur5_rviz/src/utils')
from datareader import DataReader

In [8]:
fns = ['Knot_Tying_D001.txt','Knot_Tying_D002.txt','Knot_Tying_D003.txt','Knot_Tying_D004.txt','Knot_Tying_D005.txt','Knot_Tying_E001.txt','Knot_Tying_E002.txt','Knot_Tying_E003.txt','Knot_Tying_E004.txt','Knot_Tying_E005.txt','Needle_Passing_D001.txt','Needle_Passing_D002.txt','Needle_Passing_D003.txt','Needle_Passing_D004.txt','Needle_Passing_D005.txt','Needle_Passing_E001.txt','Needle_Passing_E002.txt','Needle_Passing_E003.txt','Needle_Passing_E004.txt','Needle_Passing_E005.txt','Suturing_D001.txt','Suturing_D002.txt','Suturing_D003.txt','Suturing_D004.txt','Suturing_D005.txt','Suturing_E001.txt','Suturing_E002.txt','Suturing_E003.txt','Suturing_E004.txt','Suturing_E005.txt']

In [9]:
arms = [True,False]
pbar = tqdm(total=len(fns)*2)

dscores = {}

for fn in fns:
    for arm in arms:
        folder = "../assets/dataset/"
        path = os.path.join(folder, fn)
        dr = DataReader(path)
        dr.read_dataset(arm)
        dr.calculate()

        tmp_urdf_path = "/home/emanuelsamir/Documentos/dev/concytec/journal_ws/src/ur5_description/urdf/ur5_uncertainty_0.urdf"
        dt = 0.01
        robot_tmp = Robot(np.zeros(6), np.zeros(6), dt, tmp_urdf_path)

        x0, _,_,_ = dr.dataset_trajectory_generator()
        p0 = x0[0:3]
        rpy0 = x0[3:6]

        q_des= robot_tmp.inverse_kinematics_pose(p0, rpy2rot(rpy0), np.zeros(6), max_iter = 1000) 
        q_des = np.arctan2(np.sin(q_des), np.cos(q_des))#q_des = np.clip(q_des, a_min=-np.pi, a_max = np.pi)

        qs = []

        while True:
            x_des, dx_des, ddx_des, dddx_des = dr.dataset_trajectory_generator()
            p_des = x_des[0:3]
            rpy_des = x_des[3:6]

            q = robot_tmp.inverse_kinematics_pose(p_des, rpy2rot(rpy_des), q_des)
            qs.append(q)
            if dr.check():
                break 
            q_des = q
        qs_ar = np.array(qs)
        pbar.update(1)

        if np.max(qs_ar) < np.pi and np.min(qs_ar) > -np.pi:
            name = 'file {} arm {}'.format(fn, 'r' if arm else 'l')
            score = np.mean([np.pi - np.max(qs_ar), np.pi + np.min(qs_ar) ])
            dscores[name] = score

pbar.close()

100%|██████████| 60/60 [09:01<00:00,  9.03s/it]


In [7]:
import operator
sorted_x = sorted(dscores.items(), key=operator.itemgetter(1))
print(sorted_x)

[('file Knot_Tying_E001.txt arm r', 0.1997178283127765), ('file Needle_Passing_E004.txt arm l', 0.31442422649038027), ('file Knot_Tying_D003.txt arm l', 0.3162118185579512), ('file Suturing_E004.txt arm r', 0.3261896858179627), ('file Knot_Tying_E003.txt arm r', 0.385570999034367), ('file Suturing_E005.txt arm r', 0.45083115345879654), ('file Knot_Tying_E002.txt arm l', 0.5536733973592689), ('file Knot_Tying_D001.txt arm l', 0.6464879566011885), ('file Needle_Passing_E002.txt arm l', 0.6950999912380562), ('file Knot_Tying_E005.txt arm l', 0.7387523430119407), ('file Knot_Tying_D003.txt arm r', 0.7425371592375132), ('file Knot_Tying_D002.txt arm l', 0.7675061186538925)]


In [6]:
sorted(dscores.items())

[('file Knot_Tying_D001.txt arm l', 0.7379998121805957),
 ('file Knot_Tying_D002.txt arm l', 0.48706938835816826),
 ('file Knot_Tying_D002.txt arm r', 0.23171329482667513),
 ('file Knot_Tying_D003.txt arm l', 0.3883294562379771),
 ('file Knot_Tying_D003.txt arm r', 0.3842611100576778),
 ('file Knot_Tying_D004.txt arm r', 1.1231724805685528),
 ('file Knot_Tying_D005.txt arm l', 0.7423733582672545),
 ('file Knot_Tying_E001.txt arm l', 0.10484600339984396),
 ('file Knot_Tying_E002.txt arm r', 0.6116607544616999),
 ('file Knot_Tying_E004.txt arm l', 0.344820567481847),
 ('file Knot_Tying_E004.txt arm r', 0.6342137650208912),
 ('file Knot_Tying_E005.txt arm l', 0.31790024519795157),
 ('file Knot_Tying_E005.txt arm r', 1.0149287145583408),
 ('file Needle_Passing_D001.txt arm l', 1.4444813142015274),
 ('file Needle_Passing_D003.txt arm l', 1.4249336430369466),
 ('file Needle_Passing_D004.txt arm l', 1.2152288414553292),
 ('file Needle_Passing_D005.txt arm l', 0.3607657550619705),
 ('file Need